# Data Understanding (NO2 Kabupaten Sidoarjo)
## 1.1 Sumber Data
Data NO₂ diperoleh dari Copernicus / Sentinel-5P melalui endpoint openEO (```openeo.dataspace.copernicus.eu```) seperti yang terlihat pada skrip Python yang diberikan. Dalam alur yang dipakai:
- Koleksi yang dimuat: ```SENTINEL_5P_L2```.
- Rentang temporal: ```["2022-10-01", "2025-10-01"]``` (contoh pada kode).
- Area of interest (AOI): sebuah poligon kecil di sekitar koordinat longitude ~110.37–110.48 dan latitude ~ -6.94– -7.03.
- Band yang diminta: ```NO2``` (komponen NO₂ dari produk pemantauan atmosfer Sentinel-5P).
- Agregasi yang dilakukan:
    - ```mask``` untuk menandai nilai invalid (negatif) sebagai tidak valid,
    - ```aggregate_temporal_period(period="day", reducer="mean")``` → rata-rata harian,
    - ```aggregate_spatial(geometries=aoi, reducer="mean")``` → rata-rata spasial di atas AOI,
    - hasil diekspor batch ke CSV.

In [1]:
import requests
import openeo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
import os

# connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()
# print("Connected to Copernicus Data Space Ecosystem")

session = requests.Session()
session.timeout = 300
connection = openeo.connect("openeo.dataspace.copernicus.eu", session=session).authenticate_oidc()
print("Connected to Copernicus Data Space Ecosystem")

ModuleNotFoundError: No module named 'openeo'

In [18]:
aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              112.48915779619239,
              -7.330985416832718
            ],
            [
              112.48915779619239,
              -7.578877834418705
            ],
            [
              112.85189705981338,
              -7.578877834418705
            ],
            [
              112.85189705981338,
              -7.330985416832718
            ],
            [
              112.48915779619239,
              -7.330985416832718
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

s5p = connection.load_collection(
    "SENTINEL_5P_L2",
    spatial_extent={
        "west": 112.48915779619239,
        "south": -7.578877834418705,
        "east": 112.85189705981338,
        "north": -7.330985416832718,
    },
    temporal_extent=["2022-10-01", "2025-10-01"],
    bands=["NO2"],
)


def mask_invalid(x):
    return x < 0

s5p_masked = s5p.mask(s5p.apply(mask_invalid))

daily_mean = s5p_masked.aggregate_temporal_period(period="day", reducer="mean")

daily_mean_aoi = daily_mean.aggregate_spatial(geometries=aoi, reducer="mean")

job = daily_mean_aoi.execute_batch(out_format="CSV")

results = job.get_results()
results.download_files("data-copernicus")

for f in os.listdir("data-copernicus"):
    if f.endswith(".csv"):
        df = pd.read_csv(os.path.join("data-copernicus", f))
        print("File ditemukan:", f)
        break
    
df["date"] = pd.to_datetime(df["date"])

df["month"] = df["date"].dt.to_period("M")

df_monthly = df.groupby("month", as_index=False)["NO2"].mean()
df




0:00:00 Job 'j-25102414322040478e7ad8db67c95fc5': send 'start'
0:00:14 Job 'j-25102414322040478e7ad8db67c95fc5': created (progress 0%)
0:00:19 Job 'j-25102414322040478e7ad8db67c95fc5': created (progress 0%)
0:00:26 Job 'j-25102414322040478e7ad8db67c95fc5': created (progress 0%)
0:00:34 Job 'j-25102414322040478e7ad8db67c95fc5': created (progress 0%)
0:00:44 Job 'j-25102414322040478e7ad8db67c95fc5': created (progress 0%)
0:00:57 Job 'j-25102414322040478e7ad8db67c95fc5': running (progress N/A)
0:01:13 Job 'j-25102414322040478e7ad8db67c95fc5': running (progress N/A)
0:01:32 Job 'j-25102414322040478e7ad8db67c95fc5': running (progress N/A)
0:01:56 Job 'j-25102414322040478e7ad8db67c95fc5': running (progress N/A)
0:02:27 Job 'j-25102414322040478e7ad8db67c95fc5': running (progress N/A)
0:03:04 Job 'j-25102414322040478e7ad8db67c95fc5': running (progress N/A)
0:03:51 Job 'j-25102414322040478e7ad8db67c95fc5': running (progress N/A)
0:04:50 Job 'j-25102414322040478e7ad8db67c95fc5': running (progres

C:\Users\Harseven\AppData\Local\Temp\ipykernel_4640\1944329125.py:73: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["date"].dt.to_period("M")


,date,feature_index,NO2,month
0,2022-10-02 00:00:00+00:00,0,0.000048,2022-10
1,2022-10-01 00:00:00+00:00,0,NaN,2022-10
2,2022-10-05 00:00:00+00:00,0,NaN,2022-10
3,2022-10-04 00:00:00+00:00,0,NaN,2022-10
4,2022-09-30 00:00:00+00:00,0,NaN,2022-09
...,...,...,...,...
1092,2025-01-09 00:00:00+00:00,0,0.000029,2025-01
1093,2025-01-15 00:00:00+00:00,0,NaN,2025-01
1094,2025-01-11 00:00:00+00:00,0,0.000058,2025-01
1095,2025-01-14 00:00:00+00:00,0,NaN,2025-01


## 1.2 Struktur Kolom (kolom yang muncul berdasarkan cuplikan & kode)
Setelah job dieksekusi dan CSV dibaca ke pandas.DataFrame, kolom yang tampak pada cuplikan adalah:

date

Tipe: datetime (pada kode lalu dikonversi pd.to_datetime(df["date"])).

Format contoh: 2024-10-12 00:00:00+00:00 → termasuk offset zona waktu (+00:00 / UTC).

Arti: tanggal tengah periode agregasi (rata-rata harian). Karena agregasi period="day", setiap baris mewakili nilai rata-rata NO₂ untuk hari tersebut.

feature_index

Tipe: integer (contoh: 0 pada banyak baris).

Arti kemungkinan: indeks fitur/patch/pixel tile dari koleksi openEO (bila batch menghasilkan beberapa fitur/spatial tiles). Karena kamu meng-aggregate_spatial menggunakan satu AOI, nilai ini sering tetap 0. Bila pipeline menghasilkan beberapa spatial features, nilai ini membantu membedakan feature geometry.

NO2

Tipe: float (angka desimal kecil, atau NaN bila tidak ada pengukuran valid hari itu).

Arti: nilai rata-rata NO₂ hasil agregasi spasial+temporal untuk AOI pada tanggal tersebut.

Rentang: pada cuplikan terlihat sangat kecil (mis. 0.000033, 0.000054), menandakan unit kolom kepadatan (perlu verifikasi metadata).

NaN: menandakan tidak ada data valid (mis. semua piksel negatif/invalid pada hari itu, atau tidak ada pengamatan yang lolos mask).

Contoh Table Data :

In [19]:
new_df = pd.read_csv('data-copernicus/timeseries.csv')
new_df = new_df.sort_values(by='date').reset_index(drop=True)
new_df['date'] = pd.to_datetime(new_df['date']).dt.date
new_df

,date,feature_index,NO2
0,2022-09-30,0,NaN
1,2022-10-01,0,NaN
2,2022-10-02,0,0.000048
3,2022-10-03,0,0.000040
4,2022-10-04,0,NaN
...,...,...,...
1092,2025-09-26,0,0.000030
1093,2025-09-27,0,0.000036
1094,2025-09-28,0,0.000015
1095,2025-09-29,0,0.000031


## 1.3 Indentifikasi Missing Value

In [20]:
missing_count = new_df.isnull().sum()
missing_percent = (missing_count / len(new_df)) * 100

missing_table = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percent (%)': missing_percent.round(2)
})

missing_table

,Missing Count,Missing Percent (%)
date,0,0.00
feature_index,0,0.00
NO2,254,23.15


Berdasarkan hasil pemeriksaan terhadap dataset yang diperoleh dari platform Copernicus Open Data (Sentinel-5P L2), terdapat tiga kolom utama yaitu:

date — berisi informasi tanggal pengambilan data, tanpa nilai kosong (0 missing value).

feature_index — kolom indeks fitur yang digunakan sistem, tanpa nilai kosong (0 missing value).

NO2 — berisi konsentrasi Nitrogen Dioksida (NO₂) rata-rata harian pada area pengamatan, dengan 254 nilai yang hilang (missing values).

Kehilangan data pada kolom NO₂ kemungkinan disebabkan oleh beberapa faktor seperti gangguan atmosfer, awan tebal, atau noise dalam citra satelit Sentinel-5P yang menghambat pembacaan sensor.